In [50]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
import pandas as pd
from surprise import Dataset
from surprise import Reader

reader = Reader(rating_scale=(1,5))

# Load the movielens-100k dataset (download it if needed).
svd = SVD()

df = pd.read_csv('winequality-red.csv')


df.head()

data = Dataset.load_from_df(df[['sulphates', 'alcohol', 'quality']],
reader)


# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9893  1.0269  1.0359  1.0166  1.0720  1.0281  0.0269  
MAE (testset)     0.6821  0.7205  0.7571  0.7237  0.7744  0.7315  0.0320  
Fit time          0.01    0.01    0.01    0.01    0.01    0.01    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([0.98925094, 1.02688478, 1.03590462, 1.01657786, 1.07199212]),
 'test_mae': array([0.68205606, 0.72048847, 0.75709847, 0.72369709, 0.77436806]),
 'fit_time': (0.01322484016418457,
  0.010385990142822266,
  0.0087890625,
  0.0075109004974365234,
  0.006757259368896484),
 'test_time': (0.001773834228515625,
  0.001689910888671875,
  0.0015039443969726562,
  0.00127410888671875,
  0.0011317729949951172)}

In [51]:
from surprise import SVD
from surprise.model_selection import GridSearchCV
 
param_grid = {
  'n_factors': [20, 50, 100],
  'n_epochs': [5, 10, 20]
}
 
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=10)
gs.fit(data)
 
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

1.027380021650759
{'n_factors': 20, 'n_epochs': 5}


In [52]:
from surprise.model_selection import train_test_split
 
# best hyperparameters
best_factor = gs.best_params['rmse']['n_factors']
best_epoch = gs.best_params['rmse']['n_epochs']
 
# sample random trainset and testset
# test set is made of 20% of the ratings.
trainset, testset = train_test_split(data, test_size=.20)
 
# We'll use the famous SVD algorithm.
svd = SVD(n_factors=best_factor, n_epochs=best_epoch)
 
# Train the algorithm on the trainset
svd.fit(trainset)

In [53]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [54]:
df_title = pd.read_csv('winequality-red.csv')
df_title.head(10)
df_title.head(10)

titles = df_title.copy()
titles['Estimate_Score'] = titles['quality'].apply(lambda x: svd.predict('quality', x).est)
                                                
                                                   
                    

In [55]:
titles = titles.sort_values(by=['Estimate_Score'], ascending=False)
titles.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Estimate_Score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,5
1074,7.5,0.77,0.20,8.1,0.098,30.0,92.0,0.99892,3.20,0.58,9.2,5,5
1072,7.2,0.35,0.26,1.8,0.083,33.0,75.0,0.99680,3.40,0.58,9.5,6,5
1071,7.5,0.77,0.20,8.1,0.098,30.0,92.0,0.99892,3.20,0.58,9.2,5,5
1070,9.3,0.33,0.45,1.5,0.057,19.0,37.0,0.99498,3.18,0.89,11.1,7,5


In [56]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
